In [1]:
import pyodbc as py
import pandas as pd

In [2]:
print(py.drivers())

['SQL Server', 'Oracle in OraDB23Home1', 'ODBC Driver 17 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


In [3]:
# DATABASE MUST EXIST BEFORE RUNNING - RUN SQL FILE IN SQL SERVER FIRST
def connect_mssql(): # for VM, driver should be {SQL Server}, replace server with the one provided by SLU
    conn_str = r'''
        Driver={ODBC Driver 17 for SQL Server};
        Server=YogaPro9i\LOCAL_MSSQL;
        database=lab3_group17;
        trusted_connection=yes;
        '''
    conn = py.connect(conn_str)
    print(f'Connected to SQL Server')
    cur = conn.cursor()
    return conn, cur

def disconnect_mssql(conn):
    conn.close()
    print('Disconnected')

### the following function will allow us to easily insert records into the different tables

In [4]:
# call within dml functions that have already established conn and cur to track total records
def select_count(cur, table, count_type, cmd_type):
    query = f'select count(*) from {table};'
    count = cur.execute(query).fetchone()[0]
    print(f'Records in table {count_type} {cmd_type}: {count}')
    return count

# example call: (table='general.statedetail', fields=('StateID, State'), values=states')
def insert_into_database(table, fields, values):
    # list of '?' for 'values' placeholders for executemany func, use with params to create query 
    values_ph = ('?, ' * len(fields.split(',')))[:-2] # remove the last comma
    insert_query = f'insert into {table} ({fields}) values ({values_ph});'
    
    try:
        # connect, establish cursor, get and print pre-count
        conn, cur = connect_mssql()
        pre_count = select_count(cur, table, 'before', 'insert')
        
        # insert records
        cur.executemany(insert_query, values)
        
        # get and print post-count, calculate number of records inserted
        post_count = select_count(cur, table, 'after', 'insert')
        new_recs = post_count - pre_count
        
        # if number new recs equals len of values parameter, commit
        if new_recs == len(values):
            conn.commit()
            print(f'{new_recs} new records inserted in {table}')
        else:
            print('Not committed, all records were not inserted')
            
        disconnect_mssql(conn)
        
    except Exception as e:
        if conn:
            disconnect_mssql(conn)      
        print(e)
        
def delete_from_table(table):
    query = f'delete from {table};'
    conn, cur = connect_mssql()
    select_count(cur, table, 'before', 'delete')
    cur.execute(query)
    cur.commit()
    select_count(cur, table, 'after', 'delete')
    disconnect_mssql(conn)

In [15]:
# delete statement for testing
#delete_from_table('general.statedetail')
delete_from_table('hr.employee')

Connected to SQL Server
Records in table before delete: 1
Records in table after delete: 0
Disconnected


In [6]:
# insert states into statedetail table
states = [
    ('AL', 'Alabama'), ('AK', 'Alaska'), ('AZ', 'Arizona'), ('AR', 'Arkansas'),
    ('CA', 'California'), ('CO', 'Colorado'), ('CT', 'Connecticut'),
    ('DE', 'Delaware'), ('FL', 'Florida'), ('GA', 'Georgia'), ('HI', 'Hawaii'),
    ('ID', 'Idaho'), ('IL', 'Illinois'), ('IN', 'Indiana'), ('IA', 'Iowa'),
    ('KS', 'Kansas'), ('KY', 'Kentucky'), ('LA', 'Louisiana'), ('ME', 'Maine'),
    ('MD', 'Maryland'), ('MA', 'Massachusetts'), ('MI', 'Michigan'),
    ('MN', 'Minnesota'), ('MS', 'Mississippi'), ('MO', 'Missouri'),
    ('MT', 'Montana'), ('NE', 'Nebraska'), ('NV', 'Nevada'),
    ('NH', 'New Hampshire'), ('NJ', 'New Jersey'), ('NM', 'New Mexico'),
    ('NY', 'New York'), ('NC', 'North Carolina'), ('ND', 'North Dakota'),
    ('OH', 'Ohio'), ('OK', 'Oklahoma'), ('OR', 'Oregon'),
    ('PA', 'Pennsylvania'), ('RI', 'Rhode Island'), ('SC', 'South Carolina'),
    ('SD', 'South Dakota'), ('TN', 'Tennessee'), ('TX', 'Texas'), ('UT', 'Utah'),
    ('VT', 'Vermont'), ('VA', 'Virginia'), ('WA', 'Washington'),
    ('WV', 'West Virginia'), ('WI', 'Wisconsin'), ('WY', 'Wyoming')
]

#insert_into_database(table='general.statedetail', fields=('StateID, State'), values=states)

In [ ]:
depts = [
    ('GROC', 'Grocery'), ('HOME', 'Home Goods'), ('TECH', 'Electronics'), 
    ('PETS', 'Pet Supplies'), ('APPL', 'Applicances'), ('SPRT', 'Sporting Goods'),
    ('ETMT', 'Entertainment/Toys'), ('GNRL', 'General')
]

#insert_into_database(table='hr.department', fields='DeptID, Department', values=depts)

Connected to SQL Server
Records in table before insert: 0
Records in table after insert: 8
8 new records inserted in hr.department
Disconnected


In [26]:
# TODO - create function for assigning a new numeric id, which can be used with inserting

def get_last_id(table, id_type):
    last_id = None
    query = f'select max(a.{id_type}) from {table} a'
    conn, cur = connect_mssql()
    last_id = cur.execute(query).fetchone()[0]
    disconnect_mssql(conn)
    return last_id
    
def assign_id(table, id_type, num_digs, num_ids):
    last_id = get_last_id(table, id_type)

    # if table doesn't already have at least one id, create one based on the num_digs
    if not last_id:
        last_id = int('1' + ('0' * (num_digs - 1)))
        
    new_ids = []
    for i in range(num_ids):
        new_id = last_id + 1
        new_ids.append(new_id)
        last_id += 1
    
    return new_ids

assign_id('hr.employee', 'EmployeeID', 3, 15)


# TODO - now figure out how to insert sample data along with the ids    

Connected to SQL Server
Disconnected


[101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115]